- author: Lee Meng
- date: 2019-09-21 17:22
- title: 透過 Manim 動畫直觀理解神經網路 & 線性代數
- slug: understand-neural-net-and-linear-algebra-using-manim
- tags: Manim, Python
- summary: s
- description: s 
- image: s
- status: draft

!quote
- 這是篇透過動畫來闡述神經網路基本概念的科普文。讀完本文，你將能夠直觀地理解神經網路以及其與線性代數之間的緊密關係，並進一步展開自己的學習之旅。

（小提醒：本文適合以**暗色模式**閱讀。你可以點選畫面左下按鈕切換模式）

這是個眾人對人工智慧（**A**rtificial **I**ntelligence, AI）趨之若鶩的時代。此領域近年的快速發展很大一部份可歸功於[深度學習](https://leemeng.tw/deep-learning-resources.html)以及[人工神經網路（**N**eural **N**etwork, 後簡稱為 NN）](https://zh.wikipedia.org/wiki/%E4%BA%BA%E5%B7%A5%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C)。深度學習框架也日漸成熟，讓任何人都可以輕鬆地使用 [TensorFlow](https://colab.research.google.com/github/tensorflow/docs/blob/r2.0rc/site/en/r2/tutorials/quickstart/beginner.ipynb) 或 [PyTorch](https://pytorch.org/) 在 30 秒內就訓練出一個具有 98% 正確率的阿拉伯數字辨識應用。

以下簡短的程式碼就能訓練出一個能夠辨別數字的神經網路：

```python
# 此例使用 TensorFlow，但各大深度學習框架的實現邏輯基本上類似
import tensorflow as tf

# 載入深度學習 Hello World: MNIST 數字 dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# 建立一個具有 10 萬參數的神經網路（模型）
# 在現在模型參數動輒上千萬的年代，此神經網路可說是小到不行
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

# 選擇 loss func、optimizer 並訓練模型
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

# 訓練後的 NN 在測試集上可得到近 98% 正確辨識率
model.evaluate(x_test, y_test)

# 測試結果
# loss: 0.0750 - accuracy: 0.9763
```

儘管擁有近 10 萬個可訓練的參數，透過 [Keras](https://www.tensorflow.org/guide/keras) 定義的這個 `model` 是個在深度學習領域裡會被視為 Hello World 等級的**簡單**神經網路。

但就算只是一個這樣的**簡單** NN，我相信仍有不少人「見山不是山」：很多人會使用框架的 API 來建立神經網路，但並不深刻地了解其內部的運作機制。而之所以會有這樣的現象，主要是因為：
- 強大的深度學習框架把底層細節邏輯都包了起來
- 趕鴨子上桌，沒搞懂跟 NN 相關的[線性代數](https://www.youtube.com/watch?v=fNk_zzaMoSs&list=PLZHQObOWTQDPD3MizzM2xVFitgF8hE_ab)概念
- 能視覺化簡單 NN 內部機制的手法還不那麼普遍

!image
- manim/mnist-simple.jpg
- TODO: 加上 dropout 以及 forward 動畫

為了應付上述這些問題，在接下來的章節我將結合：
1. 繪圖工具：能用來建立動畫的強大 Python 函式庫 [Manim](https://github.com/3b1b/manim)
2. 學習任務：比 [MNIST](https://en.wikipedia.org/wiki/MNIST_database) 還簡單的[二分類（Binary Classification）](https://en.wikipedia.org/wiki/Binary_classification)問題
3. 模型架構：1 到 2 層、只有不到 10 個參數的超級簡單神經網路

3 者來嘗試說明最基本的 NN 概念以及相關的線性代數（Linear Algebra）知識。我將運用動畫（而非嚴謹的數學證明）來讓更多想要開始接觸 AI 及深度學習領域的你直觀地理解神經網路，並順利地踏入深度學習這塊領域。如果這聽起來還不錯，那就繼續往下閱讀吧！

## 第一印象：直觀感受簡單 NN 如何解決二分類任務

你的時間寶貴，因此我先將本文精華都濃縮在底下的 1 分鐘短片了。該影片視覺化一個簡單神經網路解決二分類任務的過程。在你看完之後我會針對影片內容下些評論，但現在請你馬上點擊下方影片的播放鍵吧！

!mp4
- options: no-loop, no-autoplay, controls
- images/manim/TwoLayersReLUInBetweenSolveHardTwoCurves.mp4